In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train =  pd.read_csv("/kaggle/input/part-3df/train.csv")
val = pd.read_csv("/kaggle/input/val-emotions/validation-set-sample.csv")

In [4]:
labels = ["Joy", "Fear", "Anger", "Sadness", "Disgust", "Surprise"]

train["No"] = ((train[labels].to_numpy() == 0).all(axis=1)).astype(int)
val["No"] = ((val[labels].to_numpy() == 0).all(axis=1)).astype(int)

In [5]:
labels = ["Joy", "Fear", "Anger", "Sadness", "Disgust", "Surprise", "No"]

x_train, x_val = train["text"].astype(str).tolist(), val["text"].astype(str).tolist()

y_train = (train[labels].astype(float).values>0).astype(float).tolist()
y_val = (val[labels].astype(float).values>0).astype(float).tolist()

In [6]:
from datasets import Dataset

train_ds = Dataset.from_dict({"text": x_train, "labels": y_train})
val_ds = Dataset.from_dict({"text": x_val, "labels": y_val})

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokens = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base", use_fast=True)

def tok_fn_base(batch):
    return tokens(batch["text"], truncation=True)

train_befor = train_ds.map(tok_fn_base, batched=True)
val_befor = val_ds.map(tok_fn_base, batched=True)

collator_befor = DataCollatorWithPadding(tokenizer=tokens)

2026-02-02 15:21:08.593881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770045668.784073      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770045668.842672      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770045669.316407      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770045669.316443      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770045669.316446      55 computation_placer.cc:177] computation placer alr

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/2466 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [8]:
from sklearn.metrics import classification_report

def sigmoid(x):
   return 1/(1 + np.exp(-x))
    
def compute_metrics(eval_pred):
    predictions, y_true = eval_pred.predictions, eval_pred.label_ids
    y_pred = sigmoid(predictions)
    y_pred = (y_pred>0.2).astype(float)
    y_true = (y_true>0).astype(float)
        
    clf_dict = classification_report(
        y_true, y_pred, 
        target_names=["Joy", "Fear", "Anger", "Sadness", "Disgust", "Surprise", "No"],
        zero_division=0, 
        output_dict=True)

    accuracy_thresh = float(np.mean(np.all(y_pred == y_true, axis=1)))
    return {"accuracy_thresh": accuracy_thresh, 
            "micro f1": clf_dict['micro avg']['f1-score'], 
            "macro f1": clf_dict['macro avg']['f1-score']}

In [9]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

baseline_scf = AutoModelForSequenceClassification.from_pretrained(
    "FacebookAI/xlm-roberta-base",
    num_labels=7,
    problem_type="multi_label_classification"
).to(device)

args_befor = TrainingArguments(
    output_dir="/kaggle/working/emotions",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5, # 0.0001
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=20,
    weight_decay=0.005,
    report_to="none",
    seed=32,
    fp16=torch.cuda.is_available(),
)

trainer_befor = Trainer(
    model=baseline_scf,
    args=args_befor,
    train_dataset=train_befor,
    eval_dataset=val_befor,
    tokenizer=tokens,
    data_collator=collator_befor,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print("Befor Zett")
trainer_befor.train()
baseline_eval = trainer_befor.evaluate()
baseline_eval

Befor Zett


Epoch,Training Loss,Validation Loss,Accuracy Thresh,Micro f1,Macro f1
1,No log,0.364923,0.325301,0.485149,0.143802
2,No log,0.343522,0.493976,0.488095,0.094470
3,No log,0.342285,0.493976,0.488095,0.094470
4,No log,0.336262,0.493976,0.488095,0.094470
5,No log,0.331439,0.425703,0.515679,0.158820
6,No log,0.322061,0.465863,0.509091,0.153721
7,No log,0.313549,0.409639,0.520270,0.192479
8,No log,0.302179,0.397590,0.515599,0.222846
9,No log,0.288793,0.437751,0.558773,0.331731
10,No log,0.291200,0.413655,0.546053,0.320870


{'eval_loss': 0.28773757815361023,
 'eval_accuracy_thresh': 0.44176706827309237,
 'eval_micro f1': 0.5876460767946577,
 'eval_macro f1': 0.38998947011518376,
 'eval_runtime': 0.5611,
 'eval_samples_per_second': 443.78,
 'eval_steps_per_second': 3.564,
 'epoch': 20.0}

---

In [11]:
{'eval_loss': '0.28',
 'eval_accuracy_thresh': '0.43',
 'eval_micro f1': '0.57',
 'eval_macro f1': '0.37',
 'eval_runtime': '0.60',
 'eval_samples_per_second': '418.11',
 'eval_steps_per_second': '3.36',
 'epoch': '20.00'}

{'eval_loss': '0.28',
 'eval_accuracy_thresh': '0.43',
 'eval_micro f1': '0.57',
 'eval_macro f1': '0.37',
 'eval_runtime': '0.60',
 'eval_samples_per_second': '418.11',
 'eval_steps_per_second': '3.36',
 'epoch': '20.00'}